In [ ]:
# First, parse our California school data from a CSV bundled in the repo
import csv,codecs

i = lambda x: int(x.replace(',','')) # Quick function to make "123,123" into an int 123123
cupc_csv_file = 'data/calpads_school_level_1718.csv'
schools_data = [
    r for r in 
    csv.DictReader(codecs.open(cupc_csv_file)) 
    if i(r['total_enrolled']) > 0 # ignore 0 student schools
] 

len(schools_data)

In [ ]:
from strategies.base import CEPSchool, CEPDistrict
districts = {}
for row in schools_data:
    school = CEPSchool(row)
    district = CEPDistrict(school.district,school.district_code)
    districts.setdefault(school.district,district)
    districts[school.district].schools.append(school)
districts = list(districts.values())
districts.sort()
len(districts)

In [ ]:
# The CEPDistrict class represents a School District
# it has a list of CEPSchools, and provides some aggregate information

# Let's inspect a district
san_diego = [d for d in districts if d.name == 'San Diego Unified'][0]
san_diego.name,len(san_diego.schools),san_diego.code,san_diego.overall_isp,san_diego.total_enrolled

    

In [ ]:
# Strategies are the algorithms we use to group schools
# Each one extends BaseStrategy and overrides the "create_groups" method
# Here we need to define which strategies we want to run
# We pass them into parse_districts as the class name, any parameters for it, and a name
# this way we can run the sam strategy in multiple configurations

from strategies.naive import OneGroupCEPStrategy,OneToOneCEPStrategy

# If we create a strategy, and run its create_groups method with the district as the
# incoming parameter, then it fills its own "groups" member with the resulting CEPGroup objects
#strategy = OneGroupCEPStrategy()
strategy = OneToOneCEPStrategy()
strategy.create_groups(san_diego)
g = strategy.groups[0] # our naive strategy put all the schools into one CEPGroup
g,g.covered_students

In [ ]:
# We can even see the estimated reimbursement range (based upon configurable paramters)
g.est_reimbursement()